In [ ]:
import requests
import csv
import pandas as pd
import json
import ast
import fastf1
import fastf1.plotting

In [ ]:
rounds_df = pd.read_csv('../data/cleaned_circuits_2000-2024.csv')
rounds_from_2018_df = rounds_df[rounds_df['season'] >= 2024]
laps_list = []
drivers_list = []
results_list = []

for _, row in rounds_from_2018_df.iterrows():
    season = row['season']
    round_name = row['round']

    session = fastf1.get_session(season, round_name, 'R')
    session.load()

    laps = session.laps.copy()
    laps['Season'] = season
    laps['Round'] = round_name
    laps_list.append(laps)

    # drivers = pd.DataFrame.from_dict(session.drivers, orient='index').copy()
    # drivers['Season'] = season
    # drivers['Round'] = round_name
    # drivers_list.append(drivers)

    results = session.results
    results['Season'] = season
    results['Round'] = round_name
    results_list.append(results)

laps_df = pd.concat(laps_data, ignore_index=True) if laps_data else pd.DataFrame()
# drivers_df = pd.concat(drivers_data, ignore_index=True) if drivers_data else pd.DataFrame()
results_df = pd.concat(results_data, ignore_index=True) if results_data else pd.DataFrame()


In [ ]:
circuits_data = pd.read_csv('../data/cleaned_circuits_2000-2024.csv')

In [ ]:
races = circuits_data[circuits_data['season'] == 2024]

In [ ]:
races.tail()

### Tyres - Stint

In [ ]:
# fastf1.Cache.enable_cache('../cache') 

session = fastf1.get_session(2022, 'Hungary', 'R')
session.load()
laps = session.laps

drivers = session.drivers

drivers = [session.get_driver(driver)["Abbreviation"] for driver in drivers]

results = session.results
positions = results[["DriverNumber", "Abbreviation", "GridPosition", "Position"]]
positions_list = positions['Abbreviation'].to_list()

stints = laps[["Driver", "DriverNumber", "Stint", "Compound", "LapNumber"]]
stints = stints.groupby(["Driver", "DriverNumber", "Stint", "Compound"])
stints = stints.count().reset_index()

stints = stints.rename(columns={"LapNumber": "StintLength"})


In [ ]:
print(session.drivers)

In [ ]:
stints.head()

In [ ]:
laps.head()

In [ ]:
laps.tail()

In [ ]:
laps_updated = laps.drop(columns=['Time', 'PitOutTime', 'PitInTime', 'FastF1Generated', 'FreshTyre', 'IsAccurate', 'Deleted', 'DeletedReason'], axis=1)

In [ ]:
laps_updated.head()

In [ ]:
sorted_positions = laps_updated.sort_values(by='Position', ascending=True)
sorted_positions.head()

In [ ]:
import plotly.express as px

# Plotting using Plotly Express
fig = px.line(laps_updated, 
            x='LapNumber', 
            y='Position', 
            color='Driver', 
            markers=True)
            
fig.update_layout(
    yaxis=dict(autorange="reversed")  # Invertir el orden del eje Y
)
# Mostrar el gráfico
fig.show()


In [ ]:
positions.head()

In [ ]:
positions['PositionChange'] = positions['GridPosition'] - positions['Position']
positions.head()

In [ ]:
position_changes = positions.groupby(['DriverNumber', 'Abbreviation'])['PositionChange'].mean().reset_index()

fig = px.bar(
    position_changes,
    x='PositionChange',
    y='Abbreviation',
    color='PositionChange',
    orientation='h',
    title="Ganancia o Pérdida de Posiciones por Piloto"
)
fig.show()

In [ ]:
# Lap times
laps_updated.head()
laps_updated['LapTimeSeconds'] = laps_updated['LapTime'].dt.total_seconds()

positions_list = positions['Abbreviation'].to_list()

fig = px.violin(
    laps_updated,
    x='Driver',
    y='LapTimeSeconds',
    category_orders={'Driver': positions_list},
    box=True,
    points="all"
)
fig.show()

In [ ]:
laps_updated.head(5)

#### Fastest laps


In [ ]:

fastest_laps = laps_updated.groupby('Driver')['LapTimeSeconds'].min().reset_index()
fig = px.bar(
    fastest_laps,
    x='Driver',
    y='LapTimeSeconds',
    color='Driver',
    category_orders={'Driver': positions_list},
    title="Vueltas Más Rápidas por Piloto",
)
fig.update_yaxes(range=[60,110])
fig.show()

## Compuestos

#### Stints

In [ ]:
stints_with_position = pd.merge(stints, positions, on=['DriverNumber'], how='left')
stints_with_position = stints_with_position.astype({'Position':'int64', 'GridPosition':'int64'})
stints_with_position = stints_with_position.sort_values(by=['Position', 'Stint'])
stints_with_position.head()

In [ ]:
import plotly.graph_objects as go
import numpy as np
# Crear una figura en Plotly
fig = go.Figure()

compounds_set = set()
# Iterar por cada piloto
for driver in drivers:

    driver_stints = stints_with_position.loc[stints["Driver"] == driver].copy()

    driver_stints[['Stint', 'Compound']] = driver_stints[['Stint', 'Compound']].replace('nan', np.nan)
    driver_stints[['Stint', 'Compound']] = driver_stints[['Stint', 'Compound']].bfill()
    
    previous_stint_end = 0
    for _, row in driver_stints.iterrows():
        # Obtener el color correspondiente al compuesto
        compound_color = fastf1.plotting.get_compound_color(row["Compound"], session=session)
        compounds_set.add((row["Compound"],compound_color))

        # Añadir una barra horizontal para cada stint del piloto
        fig.add_trace(go.Bar(
            y=[driver],  # Eje Y muestra al piloto
            x=[row["StintLength"]],  # Eje X es la duración del stint
            base=previous_stint_end,  # El inicio de la barra
            orientation='h',  # Barras horizontales
            marker=dict(color=compound_color),  # Colores y bordes
            name=row["Compound"],  # Nombre del compuesto para el hover,
            hovertemplate=(
                f"Piloto: {driver}<br>" +
                f"Compuesto: {row['Compound']}<br>" +
                f"Duración del stint: {row['StintLength']} vueltas<br>" +
                f"Inicia en la vuelta: {previous_stint_end}<br>"+
                f"Posicion inicial: {row['GridPosition']}<br>"+
                f"Posicion final: {row['Position']}<br>"
            ),
            showlegend=False 
        ))

        previous_stint_end += row["StintLength"]
    
for compound, color in compounds_set:
    fig.add_trace(go.Bar(
        y=[None],  # Para no mostrar una barra visible, solo usar la leyenda
        x=[0],  # Valor de 0 para que no se dibuje una barra visible
        name=compound,  # Nombre de la entrada de la leyenda
        marker=dict(color=color),  # El color correspondiente al compuesto
        showlegend=True  # Mostrar una entrada en la leyenda para cada compuesto
))

# Configuración del diseño
fig.update_layout(
    title="2022 Hungarian Grand Prix Strategies",
    height=800,
    xaxis_title="Lap Number",
    yaxis_title="Driver",
    barmode='stack',  # Las barras se apilan horizontalmente
    xaxis=dict(showgrid=False, zeroline=False),  # Ocultar líneas de rejilla
    yaxis=dict(autorange="reversed"),  # Invertir el orden de los pilotos
    template='plotly_white'  # Tema blanco limpio
)

# Mostrar el gráfico
fig.show()


#### Tiempo promedio de vuelta por compuesto

In [ ]:
compound_colors = {compound: color for compound, color in compounds_set}


In [ ]:
compound_df = laps_updated.groupby('Compound')['LapTimeSeconds'].mean().reset_index()
compound_df.head()

In [ ]:

fig = px.bar(
    compound_df,
    x='Compound',
    y='LapTimeSeconds',
    color='Compound',
    title=f'Tiempos promedio de vuelta por compuesto',
    labels={'LapTimeSeconds': 'Tiempo promedio (s)', 'Compound': 'Compuesto'},
    color_discrete_map=compound_colors
)

fig.update_layout(showlegend=False)
fig.show()

#### Número de vueltas promedio por compuesto

In [ ]:
laps_per_stint_compound = laps_updated.groupby(['Driver', 'Stint', 'Compound'])['LapNumber'].count().reset_index()
laps_per_stint_compound.rename({'LapNumber': 'LapCount'}, inplace=True)
laps_per_stint_compound.head()

In [ ]:
laps_compound_mean_df = laps_per_stint_compound.groupby('Compound')['LapNumber'].mean().reset_index()

In [ ]:

fig = px.bar(
    laps_compound_mean_df,
    x='Compound',
    y='LapNumber',
    color='Compound',
    title=f'Número de vueltas por compuesto y stint',
    labels={'LapCount': 'Número de vueltas', 'Compound': 'Compuesto', 'Stint': 'Tanda'},
    color_discrete_map=compound_colors
)

fig.show()

In [ ]:
### Laps x Compound
fig = px.scatter(
    laps_updated,
    x='Compound',
    y='LapTimeSeconds',
    color='Compound',
    title="Consistencia de Tiempos por Vuelta considerando el compuesto",
    labels={"LapNumber": "Número de Vuelta", "LapTimeSeconds": "Tiempo por Vuelta (segundos)"},
    color_discrete_map=compound_colors
)

fig.show()

#### Tiempo promedio de vueltas por stint

In [ ]:
stint_mean = laps_updated.groupby(['Driver', 'Stint', 'Compound'])['LapTimeSeconds'].mean().reset_index()
fig = px.bar(
    stint_mean,
    x='Driver',
    y='LapTimeSeconds',
    color='Stint',
    title='Promedio de tiempos por stint y piloto',
    labels={'LapTimeSeconds': 'Tiempo promedio (s)', 'Stint': 'Stint'},
    barmode='group',
    hover_data=['Compound'],
    category_orders={'Driver': positions_list},  # Ordenar por posiciones finales
)
fig.update_layout(
    height= 700
)
fig.show()